# Query Data with AWS Data Wrangler
* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

AWS Data Wrangler is an open-source Python package that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

In [ ]:
!pip install -q awswrangler==1.2.0

In [ ]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [ ]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

dataset (bool) – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [ ]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     filters=[("product_category", "=", "Digital_Software")],
                                     dataset=True)
df_parquet_results.shape

In [ ]:
df_parquet_results.head(5)

# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [ ]:
path = '{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                filters=[("product_category", "=", "Digital_Software")],
                                dataset=True,
                                chunked=True)

In [ ]:
print(next(chunk_iter))

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [ ]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [ ]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [ ]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database=database_name
)

In [ ]:
df.head(5)

# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [ ]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_tsv),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

In [ ]:
print(next(chunk_iter))

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();